### **Testes de Diferentes RNN's para o modelo principal**

Com base na exploração e preparação de dados realizadas na Sprint 1, conforme registrado no notebook 'preparacao_dados_modelo_principal.ipynb', obtivemos uma base inicial que permitiu implementar um modelo simples de KNN, alcançando métricas satisfatórias. Agora, na Sprint 2, o objetivo foi testar modelos mais complexos utilizando Redes Neurais Recorrentes (RNNs) e avaliar o desempenho desses modelos. A seguir, apresentamos os testes realizados, incluindo o código e as métricas correspondentes.

Bibliotecas utilizadas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dropout, Dense
from tensorflow.keras.optimizers import Adam




Preparação dos dados

In [2]:
# Importando o dataframe preparado, registrado no notebook anterior. 
# Caso seja necessário, o código para exportar esse dataframe está comentado e disponível no arquivo "preparacao_dados_modelo_principal.ipynb".
df = pd.read_csv('../data/preparacao_modelo_1.csv')
df

,KNR,ID1NAME,ID1SOK,ID1SNOK,ID1DATA,ID2NAME,ID2SOK,ID2SNOK,ID2DATA,ID718NAME,ID718SOK,ID718SNOK,ID718DATA,FALHA
0,2023-2016173,0.0,0.0,0.0,0.0,0.010733,0.010791,0.0,3.736757e-06,0.005038,0.003546,0.004405,0.000047,0.0
1,2023-2026098,0.0,0.0,0.0,0.0,0.007156,0.007194,0.0,1.868379e-07,0.002519,0.003546,0.000000,0.000000,0.0
2,2023-2026162,0.0,0.0,0.0,0.0,0.007156,0.007194,0.0,9.341893e-07,0.002519,0.003546,0.000000,0.000000,0.0
3,2023-2026175,0.0,0.0,0.0,0.0,0.007156,0.007194,0.0,1.121027e-06,0.002519,0.003546,0.000000,0.000000,0.0
4,2023-2026215,0.0,0.0,0.0,0.0,0.007156,0.007194,0.0,9.341893e-07,0.002519,0.003546,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49186,2024-2976009,0.0,0.0,0.0,0.0,0.064401,0.064748,0.0,7.374491e-04,0.000000,0.000000,0.000000,0.000000,0.0
49187,2024-2976010,0.0,0.0,0.0,0.0,0.064401,0.064748,0.0,9.554889e-04,0.000000,0.000000,0.000000,0.000000,1.0
49188,2024-2976011,0.0,0.0,0.0,0.0,0.066190,0.066547,0.0,1.438091e-03,0.000000,0.000000,0.000000,0.000000,1.0
49189,2024-2976012,0.0,0.0,0.0,0.0,0.073345,0.073741,0.0,6.931648e-02,0.000000,0.000000,0.000000,0.000000,1.0


In [3]:
# Separando as features (X) e o target (y)
X = df.drop(columns=['FALHA', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y = df['FALHA']

In [ ]:
# Separando em dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Converte X_train e X_test para arrays NumPy, caso ainda não sejam.
X_train = np.array(X_train)
X_test = np.array(X_test)

# Reestrutura X_train e X_test para ter 3 dimensões.
# A nova forma do array será (n_samples, n_features, 1)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# LSTM 

LSTM (Long Short-Term Memory) é um tipo de rede neural recorrente (RNN) projetada para lidar com o problema de "vanishing gradient" que afeta as RNNs tradicionais. Ele é especialmente eficaz para modelar sequências de dados, como séries temporais, textos ou qualquer dado onde a ordem e a dependência temporal são importantes.

A LSTM utiliza células de memória que podem armazenar e acessar informações por longos períodos de tempo. Cada célula possui três portas principais:

1. **Porta de entrada (Input Gate)**: Controla a quantidade de nova informação que deve ser armazenada na célula de memória.
2. **Porta de esquecimento (Forget Gate)**: Decide quanta informação antiga deve ser descartada da célula de memória.
3. **Porta de saída (Output Gate)**: Determina qual parte da informação armazenada na célula deve ser utilizada para gerar a saída.

Essa arquitetura permite que as LSTMs capturem dependências de longo prazo em sequências, tornando-as adequadas para tarefas como previsão de séries temporais, tradução automática, e processamento de linguagem natural (NLP).

A seguir é possível acompanhar a implementação do modelo utilizando o Keras:

In [ ]:
# Construção do modelo com LSTM
model = Sequential()

model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

In [ ]:
# Treinamento do modelo
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
#Prever os dados de teste
y_pred = model.predict(X_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes = (y_pred > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred_classes)

# Exibindo a Matriz de Confusão
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Salva o modelo treinado em um arquivo no formato HDF5.
model.save('LSTM_YN.h5')

# GRU

O GRU (Gated Recurrent Unit) é outro tipo de rede neural recorrente (RNN) que, assim como a LSTM, foi projetado para lidar com o problema de "vanishing gradient" e modelar dependências de longo prazo em sequências de dados. No entanto, o GRU é uma versão simplificada do LSTM, com uma estrutura menos complexa.

**Estrutura do GRU**

O GRU combina a célula de estado e a saída em uma única unidade e utiliza duas portas principais:

1. **Porta de atualização (Update Gate)**: Controla a quantidade de nova informação que deve ser armazenada e decide quanta informação da etapa anterior deve ser esquecida ou mantida.
2. **Porta de reset (Reset Gate)**: Decide quanta da informação anterior deve ser esquecida ao calcular a nova entrada.

Devido à sua simplicidade, as GRUs tendem a ser mais rápidas de treinar e podem ser igualmente eficazes para muitas tarefas que envolvem dados sequenciais, como previsão de séries temporais e tradução de textos. Além disso, a GRU pode ser preferida quando o objetivo é reduzir a complexidade computacional, pois ela requer menos recursos comparado ao LSTM.

Abaixo é possível acompanhar a aplicação do GRU utilizando o Keras.

In [ ]:
# Construção do modelo com GRU
model = Sequential()

model.add(GRU(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(GRU(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Treinamento do modelo
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Prever os dados de teste
y_pred = model.predict(X_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes = (y_pred > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

In [ ]:
# Salva o modelo treinado em um arquivo no formato HDF5.
model.save('GRU_YN.h5')